<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch10/section_95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 95. マルチターンのチャット
問題94で生成された応答に対して、追加で”Please give me the plural form of the word with its spelling in reverse order.”と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

### チャットテンプレートとは？
会話形式で書いたやり取りを、モデルが実際に読むプロンプトに変換するための定型フォーマットのこと

### modelのrole
- system: ルール
- user: 質問
- assistant: 生成されるもの

In [13]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
!pip install -U transformers

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [16]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
chat = [
    {"role": "system", "content": "You are a friendly chatbot who always responds in the style of a pirate"},
    {"role": "user", "content": "What do you call a sweet eaten after dinner?"},
]
max_new_tokens = 50

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)

### tokenizer.apply_chat_templateとは？
chat形式（role付きの辞書）をモデルが読める形式に変換する関数
- tokenize=False(デフォルト): これは文字列（プロンプトを返す）
```
<|system|> You are a pirate
<|user|> What do you call...
<|assistant|>
```
  -  Trueの場合はトークン化して返す
### add_generation_prompt=Trueとは？
assistantが追加されてモデルが生成したものを出力できる様になる

In [19]:
tokenize_chat = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt")
prompt_len = tokenize_chat["input_ids"].shape[-1]

In [20]:
generation_config = GenerationConfig(
    max_new_tokens=max_new_tokens,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [21]:
output = model.generate(**tokenize_chat, generation_config=generation_config)

入力も返されるので、プロンプトの長さでマスクする

In [22]:
shifted_sentence = output[0][prompt_len:]
print(shifted_sentence)

tensor([   56,   261,  1427,   258,     6, 18728,   264, 32726,    11, 36346,
           30, 98693,  1243,    11, 30276,    88,     0,   358,   387,  1440,
          258,     6,   279,  4320,   311, 55295,  3488,    13, 32269,   387,
         3137,   258,     6,   297,     6,   264,  1131,   320,    67,  2453,
          780, 18579,     8,  2564,  5701,   634,   258,     6,     0,   362])


In [23]:
answer_1 = tokenizer.decode(shifted_sentence, skip_special_tokens=True)

In [24]:
print(answer_1)

Yer lookin' fer a treasure, eh? Alright then, matey! I be knowin' the answer to yer question. Ye be talkin' o' a... (dramatic pause)...puddin'! A


In [28]:
chat.append({ "role": "assistant", "content": answer_1 })
chat.append({ "role": "user", "content": "Please give me the plural form of the word with its spelling in reverse order." })

In [30]:
for i in range(len(chat)):
  print(chat[i])

{'role': 'system', 'content': 'You are a friendly chatbot who always responds in the style of a pirate'}
{'role': 'user', 'content': 'What do you call a sweet eaten after dinner?'}
{'role': 'assistant', 'content': "Yer lookin' fer a treasure, eh? Alright then, matey! I be knowin' the answer to yer question. Ye be talkin' o' a... (dramatic pause)...puddin'! A"}
{'role': 'user', 'content': 'Please give me the plural form of the word with its spelling in reverse order.'}


In [37]:
prompt_text = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt")
prompt_len = prompt_text["input_ids"].shape[-1]

In [39]:
print(prompt_text)
print(prompt_len)

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   2705,  13806,    220,   2366,     21,    271,   2675,    527,
            264,  11919,   6369,   6465,    889,   2744,  31680,    304,    279,
           1742,    315,    264,  55066, 128009, 128006,    882, 128007,    271,
           3923,    656,    499,   1650,    264,  10437,  35661,   1306,  14177,
             30, 128009, 128006,  78191, 128007,    271,     56,    261,   1427,
            258,      6,  18728,    264,  32726,     11,  36346,     30,  98693,
           1243,     11,  30276,     88,      0,    358,    387,   1440,    258,
              6,    279,   4320,    311,  55295,   3488,     13,  32269,    387,
           3137,    258,      6,    297,      6,    264,   1131,    320,     67,
           2453,    780,  18579,  40125,   5701,    634,    258,      6,      0,
            36

In [40]:
outputs = model.generate(**prompt_text, generation_config=generation_config)

In [41]:
print(outputs)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   2705,  13806,    220,   2366,     21,    271,   2675,    527,
            264,  11919,   6369,   6465,    889,   2744,  31680,    304,    279,
           1742,    315,    264,  55066, 128009, 128006,    882, 128007,    271,
           3923,    656,    499,   1650,    264,  10437,  35661,   1306,  14177,
             30, 128009, 128006,  78191, 128007,    271,     56,    261,   1427,
            258,      6,  18728,    264,  32726,     11,  36346,     30,  98693,
           1243,     11,  30276,     88,      0,    358,    387,   1440,    258,
              6,    279,   4320,    311,  55295,   3488,     13,  32269,    387,
           3137,    258,      6,    297,      6,    264,   1131,    320,     67,
           2453,    780,  18579,  40125,   5701,    634,    258,      6,      0,
            362, 128009, 128

In [42]:
shifted_sentence = outputs[0][prompt_len:]
print(shifted_sentence)

tensor([ 87575,    387,   1390,    258,      6,    279,  39598,    297,      6,
           330,   5701,    634,    258,  15260,    449,   1202,  43529,    304,
         10134,     11,  36346,     30,  98693,   1243,     11,  30276,     88,
             0,    578,   4320,    387,   1131,    330,   5701,    634,   1354,
             1,      0, 128009])


In [43]:
answer_2 = tokenizer.decode(shifted_sentence, skip_special_tokens=True)

In [44]:
print(answer_2)

Ye be wantin' the plural o' "puddin'" with its spelling in reverse, eh? Alright then, matey! The answer be... "puddins"!
